<a href="https://colab.research.google.com/github/KJM94/Team_project/blob/main/%EB%8D%B0%EC%9D%B4%EC%BD%98_%EB%B6%80%EB%8F%99%EC%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **주택공급지표를 이용한 지역별 주택공급량 시각화**

## 목차
    - 1) 입주예정물량과 매매가격지수의 관계
    - 2) 주택값 급등에 따른 미분양주택감소+광역시별 미분양
    - 3) 주택인허가,착공주택
    - 4) 분양실적, 준공실적

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

os.chdir('/content/drive/MyDrive/Colab Notebooks/housedata')

In [3]:
import pandas as pd

한글 폰트 설정

In [4]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (10.1 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

In [5]:
build_df = pd.read_csv('건축허가현황.csv')
money_df = pd.read_csv('국내건설수주액.csv')
house_df = pd.read_csv('미분양주택현황.csv')
buildre_df = pd.read_csv('주택건설인허가실적.csv')
kbsell_df = pd.read_csv('주택매매가격지수(KB).csv')
region_df = pd.read_csv('지역별_지가변동률.csv')

주택착공실적, 주택준공실적, 주택공급실적, 입주예정물량에 대한 외부 데이터 불러오기

In [8]:
# 출처 : https://www.r-one.co.kr/rone/resis/statistics/statisticsViewer.do?menuId=HOUSE_21111 한국부동산원
checkgong_df = pd.read_excel('주택건설인허가실적,착공주택2.xlsx')
# 출처 : https://www.r-one.co.kr/rone/resis/statistics/statisticsViewer.do?menuId=LFR_11100 한국부동산원
joongong_df = pd.read_excel('분양,공급실적.xlsx')
# 출처 : https://aptgin.com/home/sub04/Sub0402.do?_menuNo=144 부동산지인
ipjoo_df = pd.read_excel('입주예정물량.xlsx')

In [9]:
build_df.head()

,Unnamed: 0,연면적-철근·철골조[㎡],연면적-조적조[㎡],연면적-목조[㎡],연면적-기타(자재별)[㎡],연면적-주거용[㎡],연면적-상업용[㎡],연면적-공업용[㎡],연면적-교육및사회용[㎡],연면적-기타(용도별)[㎡],연면적-자재별[㎡],연면적-용도별[㎡],동수-철근·철골조[동수],동수-조적조[동수],동수-목조[동수],동수-기타(자재별)[동수],동수-주거용[동수],동수-상업용[동수],동수-공업용[동수],동수-교육및사회용[동수],동수-기타(용도별)[동수],동수-자재별[동수],동수-용도별[동수]
0,2000-08-01,3898712.0,122063.0,5229.0,287185.0,1812355.0,1088979.0,687577.0,0.0,724278.0,4313189.0,4313189.0,4319.0,1216.0,58.0,1078.0,2905.0,2008.0,741.0,0.0,1017.0,6671.0,6671.0
1,2000-09-01,3055754.0,113586.0,5918.0,236449.0,1273430.0,1014900.0,702592.0,0.0,420785.0,3411707.0,3411707.0,3910.0,1128.0,70.0,793.0,2653.0,1756.0,652.0,0.0,840.0,5901.0,5901.0
2,2000-10-01,4104565.0,134648.0,8006.0,314620.0,1908535.0,1095874.0,1011477.0,0.0,545953.0,4561839.0,4561839.0,4970.0,1290.0,61.0,1183.0,3246.0,2264.0,874.0,0.0,1120.0,7504.0,7504.0
3,2000-11-01,4337103.0,108212.0,7727.0,283128.0,2415543.0,1164943.0,613744.0,0.0,541940.0,4736170.0,4736170.0,4497.0,1128.0,62.0,1088.0,2801.0,2050.0,816.0,0.0,1108.0,6775.0,6775.0
4,2000-12-01,6027008.0,161879.0,26079.0,453178.0,3450325.0,1412322.0,1069299.0,0.0,736198.0,6668144.0,6668144.0,3581.0,941.0,70.0,986.0,2080.0,1619.0,748.0,0.0,1131.0,5578.0,5578.0


In [10]:
money_df.head()

,Unnamed: 0,총수주액[백만원],공공부문[백만원],중앙정부[백만원],지방자치단체[백만원],공기업[백만원],기타공공단체[백만원],민간부문[백만원],제조업[백만원],비제조업[백만원],국내외국기관[백만원],민자[백만원],건축[백만원],주택[백만원],사무실 및 점포[백만원],공장 및 창고[백만원],관공서[백만원],기타[백만원],토목[백만원],치산 및 치수[백만원],농림 및 수산[백만원],도로 및 교량[백만원],항만 및 공항[백만원],철도 및 궤도[백만원],상하수도[백만원],발전 및 송전[백만원],토지조성[백만원],댐[백만원],기계설치[백만원],기타[백만원].1,조경공사[백만원]
0,1976-01-01,12592.0,9664.0,2037.0,3844.0,3243.0,540.0,2928.0,2434.0,494.0,NaN,NaN,2231.0,186.0,928.0,862.0,239.0,16.0,10361.0,1.0,225.0,5655.0,3122.0,0.0,35.0,953.0,97.0,0.0,239.0,34.0,NaN
1,1976-02-01,8230.0,4156.0,797.0,1954.0,1216.0,189.0,4074.0,1760.0,2314.0,NaN,NaN,3762.0,246.0,847.0,1955.0,85.0,629.0,4468.0,5.0,144.0,419.0,700.0,0.0,38.0,23.0,1973.0,0.0,691.0,475.0,NaN
2,1976-03-01,22261.0,14173.0,1864.0,4764.0,7078.0,467.0,7419.0,4060.0,3359.0,669.0,NaN,11678.0,3706.0,2217.0,3454.0,1308.0,993.0,10583.0,253.0,4107.0,3779.0,560.0,213.0,134.0,230.0,321.0,0.0,717.0,269.0,NaN
3,1976-04-01,45920.0,25729.0,11859.0,6088.0,6693.0,1089.0,20161.0,4498.0,15663.0,30.0,NaN,25276.0,13984.0,6724.0,1556.0,2403.0,609.0,20644.0,413.0,1394.0,2887.0,3989.0,801.0,1584.0,340.0,1289.0,1.0,6694.0,1252.0,NaN
4,1976-05-01,48992.0,43014.0,23978.0,10847.0,6969.0,1220.0,5775.0,2578.0,3197.0,203.0,NaN,16548.0,5521.0,2021.0,1703.0,6268.0,1035.0,32444.0,1987.0,345.0,8691.0,9655.0,647.0,7414.0,485.0,875.0,168.0,598.0,1579.0,NaN


In [11]:
house_df.head()

,Unnamed: 0,전국[호],서울[호],부산[호],대구[호],인천[호],광주[호],대전[호],울산[호],경기[호],강원[호],충북[호],충남[호],전북[호],전남[호],경북[호],경남[호],제주[호],수도권[호],세종[호]
0,2007-01-01,75616.0,697.0,8680.0,9467.0,243.0,7005.0,819.0,1180.0,3604.0,5312.0,2548.0,7449.0,5179.0,5374.0,6210.0,11772.0,77.0,4544.0,NaN
1,2007-02-01,73619.0,590.0,8653.0,9134.0,312.0,6246.0,803.0,1210.0,3361.0,5344.0,2645.0,7363.0,5098.0,5253.0,5948.0,11563.0,96.0,4263.0,NaN
2,2007-03-01,73162.0,687.0,8548.0,9189.0,266.0,5905.0,1048.0,1687.0,2579.0,5801.0,2565.0,7446.0,5184.0,4887.0,5854.0,11328.0,188.0,3532.0,NaN
3,2007-04-01,73393.0,685.0,8074.0,9009.0,219.0,5829.0,1020.0,1521.0,2488.0,5460.0,3632.0,8250.0,4812.0,4664.0,5419.0,12149.0,162.0,3392.0,NaN
4,2007-05-01,78571.0,704.0,7964.0,10888.0,362.0,7842.0,957.0,1500.0,2488.0,5212.0,4012.0,9195.0,4625.0,4640.0,6303.0,11723.0,156.0,3554.0,NaN


In [12]:
buildre_df.head()

,Unnamed: 0,전국[호],부산[호],충북[호],충남[호],대구[호],대전[호],강원[호],광주[호],경기[호],인천[호],제주[호],전북[호],전남[호],경북[호],경남[호],민간부문[호],공공부문[호],부문별[호],세종[호],서울[호],울산[호]
0,2007-01-01,12038.0,455.0,188.0,1004.0,818.0,55.0,452.0,1022.0,2647.0,69.0,132.0,888.0,206.0,698.0,1691.0,11718.0,320.0,12038.0,NaN,1530.0,183.0
1,2007-02-01,29789.0,1554.0,493.0,2310.0,3549.0,89.0,642.0,2200.0,5517.0,185.0,232.0,1490.0,700.0,2942.0,4206.0,28311.0,1478.0,29789.0,NaN,2324.0,1356.0
2,2007-03-01,49827.0,3675.0,1187.0,4200.0,4596.0,154.0,1031.0,3547.0,8705.0,1115.0,337.0,2153.0,1091.0,5256.0,6527.0,48285.0,1542.0,49827.0,NaN,4212.0,2041.0
3,2007-04-01,69013.0,4039.0,1651.0,4901.0,5110.0,235.0,2185.0,3773.0,12692.0,4452.0,485.0,2627.0,2310.0,6480.0,7576.0,67439.0,1574.0,69013.0,NaN,6175.0,4322.0
4,2007-05-01,100210.0,4936.0,2331.0,7988.0,5685.0,746.0,2698.0,5844.0,25860.0,4663.0,563.0,3542.0,2663.0,7330.0,12754.0,96172.0,4038.0,100210.0,NaN,8041.0,4566.0


In [13]:
kbsell_df.head()

,Unnamed: 0,총지수[2019.01=100],단독주택[2019.01=100],연립주택[2019.01=100],아파트[2019.01=100],아파트(서울)[2019.01=100],총지수(서울)[2019.01=100]
0,1986-01-01,34.656,61.347,43.053,20.973,17.153,30.044
1,1986-02-01,34.656,61.347,42.925,20.973,17.183,30.044
2,1986-03-01,34.708,61.513,42.733,20.935,17.153,30.002
3,1986-04-01,34.449,61.097,42.733,20.819,17.034,29.837
4,1986-05-01,34.293,60.848,41.837,20.664,16.945,29.588


In [14]:
region_df.head()

,Unnamed: 0,전국[%],서울[%],부산[%],대구[%],인천[%],광주[%],대전[%],울산[%],세종[%],경기[%],강원[%],충북[%],충남[%],전북[%],전남[%],경북[%],경남[%],제주[%]
0,2005-01-01,0.226,0.192,0.083,0.212,0.145,0.156,0.196,0.137,NaN,0.374,0.142,0.221,0.463,0.108,0.099,0.163,0.140,0.150
1,2005-02-01,0.184,0.140,0.065,0.208,0.208,0.080,0.206,0.153,NaN,0.256,0.169,0.178,0.531,0.084,0.071,0.135,0.160,0.111
2,2005-03-01,0.348,0.403,0.119,0.278,0.470,0.154,0.674,0.180,NaN,0.324,0.257,0.227,1.154,0.085,0.139,0.161,0.225,0.102
3,2005-04-01,0.525,0.725,0.151,0.258,0.751,0.159,0.917,0.149,NaN,0.591,0.221,0.358,0.804,0.071,0.127,0.209,0.291,0.181
4,2005-05-01,0.562,0.590,0.185,0.254,0.693,0.291,0.850,0.131,NaN,0.882,0.229,0.476,0.883,0.094,0.152,0.224,0.310,0.247


In [15]:
checkgong_df.head()

,연도,서울(주택인허가),서울(착공주택),대전(주택인허가),대전(착공주택),인천(주택인허가),인천(착공주택),세종(주택인허가),세종(착공주택),광주(주택인허가),광주(착공주택),대구(주택인허가),대구(착공주택),울산(주택인허가),울산(착공주택),부산(주택인허가),부산(착공주택),제주(주택인허가),제주(착공주택),시별종합(주택인허가),시별종합(착공실적)
0,2013,77621,62223,5180,7669,18907,12245,17844,16501,8454,8430,18078,24303,5344,8004,29922,23248,6309,6118,187659,168741
1,2014,65249,62013,5073,7336,13583,10904,12447,16726,11056,14161,19079,28297,12502,8767,17210,29776,8805,10182,165004,188162
2,2015,101235,116450,7987,14226,30590,37316,15235,16583,14673,12182,27118,23203,12459,17168,33535,37642,18690,19234,261522,294004
3,2016,74739,81413,13509,8893,22186,15755,12627,12123,22796,17317,23169,15478,16325,14635,36664,34665,21596,20060,243611,220339
4,2017,113131,86890,9953,9828,22689,14574,9332,6269,20326,20107,31378,12482,12747,6203,47159,36968,14163,12730,280878,206051


In [16]:
joongong_df.head()

,연도,지역,실적,매매가격지수
0,2013,시별종합(분양실적),50167,89.05000
1,2013,시별종합(준공실적),173120,0.00000
2,2014,시별종합(분양실적),64652,91.16481
3,2014,시별종합(준공실적),197783,0.00000
4,2015,시별종합(분양실적),81979,94.72593


In [17]:
ipjoo_df.head()

,연도,지역,입주예정물량
0,2013,세종특별자치시,4444
1,2014,세종특별자치시,19550
2,2015,세종특별자치시,19941
3,2016,세종특별자치시,8576
4,2017,세종특별자치시,16028


## **결론**

지금까지 지표를 봤을 때 모든 지표가 대부분의 지역이 수요에 비하여 공급량이 턱없이 부족하고 그로 인해 매매가격지수는 끝없이 증가하여

어느정도 공급량이 남아있는 지역조차도 주택미보유자들이 주택을 분양 받기 힘든 상황이다.

빈익빈 부익부라고 했던가 거주를 생각하지 않고 투기로서 부동산을 바라보고 있는 사람들이 공급량을 초토화시키고, 공급량이 부족한 만큼

매매가격지수가 천정부지로 솟아올라 다음 순위 구매자들이 정작 공급량이 남아있음에도 불구하고 집을 구매하지 못하는 상황이 발생하고 있다.

또한 대출규제로 인하여 받을 수 있는 대출금은 줄었으나 집값이 많이 상승하여

자본금이 부족한 국민은 대출금을 최대로 받아도 집을 구매할 수 없는 상황들이 발생하고 있다.

따라서 부동산 정책과 규제를 정비하여 부동산 투기자들의 투기를 억제시켜야 한다.

단, 과도한 정부의 부동산 규제는 시장에 악영향을 미칠 수 있으므로 적당한 규제와 허점을 보완하는 정책을 시행함으로써 시장의 흐름에 맡겨야 된다고 생각한다